In [129]:
# imports

import openai  # for generating embeddings
import pandas as pd  # for DataFrames to store article sections and embeddings
import re  # for cutting <ref> links out of Wikipedia articles
import tiktoken  # for counting tokens

In [150]:
df = pd.read_csv("legal_acts_estonia.csv", names=['type', 'nr','text','link'])

In [151]:
df

,type,nr,text,link
0,VVS,para1,§ 1.\nVabariigi Valitsuse pädevus\n(1) Vabarii...,https://www.riigiteataja.ee/akt/VVS#para1
1,VVS,para2,§ 2.\nVabariigi Valitsuse asukoht\nVabariigi V...,https://www.riigiteataja.ee/akt/VVS#para2
2,VVS,para3,§ 3.\nVabariigi Valitsuse liikmed\n(1) Vabarii...,https://www.riigiteataja.ee/akt/VVS#para3
3,VVS,para3b1,§ 3.1.\nVabariigi Valitsuse liikme juurdepääs ...,https://www.riigiteataja.ee/akt/VVS#para3b1
4,VVS,para4,§ 4.\nVabariigi Valitsuse liikmete tööülesanne...,https://www.riigiteataja.ee/akt/VVS#para4
...,...,...,...,...
32784,NaN,para7,§ 7.\nKaitsekulu\nKaitsekulu on NATO meetodi j...,https://www.riigiteataja.ee/akt/#para7
32785,NaN,para8,§ 8.\nEesti Haigekassa eelarvepositsioon\nEest...,https://www.riigiteataja.ee/akt/#para8
32786,NaN,para9,§ 9.\nEelarveaasta jooksul riigiteede üleandmi...,https://www.riigiteataja.ee/akt/#para9
32787,NaN,para10,§ 10.\nTapa spordi- ja vabaajakeskuse ehituse ...,https://www.riigiteataja.ee/akt/#para10


In [152]:
df["text"][0]

'§\xa01.\nVabariigi Valitsuse pädevus\n(1) Vabariigi Valitsus teostab täidesaatvat riigivõimu Eesti Vabariigi põhiseaduse ja seaduste alusel.\n(2) Vabariigi Valitsus teostab täidesaatvat riigivõimu vahetult või valitsusasutuste kaudu.\n'

In [153]:
df["title"] = df["text"].str.split("\n").str[1]
df["text"] = df["text"].str.split("\n").apply(lambda x: ','.join(x[2:]))

In [154]:
df["text"] = df["text"].str.replace('\n','')

In [155]:
#have to split up to have less than 1600 words
df['split_text'] = df['text'].str.split(r'\(\d+\)')


In [156]:
df["split_text"][0]

['',
 ' Vabariigi Valitsus teostab täidesaatvat riigivõimu Eesti Vabariigi põhiseaduse ja seaduste alusel.,',
 ' Vabariigi Valitsus teostab täidesaatvat riigivõimu vahetult või valitsusasutuste kaudu.,']

In [157]:
df = df.explode("split_text")

In [158]:
df = df[df["split_text"]!= ""]

In [159]:
def count_words(text):
    return len(text.split())
df['word_count'] = df['split_text'].apply(count_words)

In [163]:
df[df["word_count"]>1000]

,type,nr,text,link,title,split_text,word_count
4917,TuMS,para13,(1) Tulumaksuga maksustatakse kõik töö eest sa...,https://www.riigiteataja.ee/akt/TuMS#para13,Palgatulu,"Tulumaksuga ei maksustata:,1) ametnikule, töö...",1059
16672,MSOS,para2,"Käesoleva seaduse tähenduses on:,1) veesõiduk ...",https://www.riigiteataja.ee/akt/MSOS#para2,Mõisted,"Käesoleva seaduse tähenduses on:,1) veesõiduk ...",1436
17195,MKS,para29,"[RT I, 23.03.2017, 1 - jõust. 01.04.2017],Maks...",https://www.riigiteataja.ee/akt/MKS#para29,Maksusaladust sisaldava teabe avaldamine,"[RT I, 23.03.2017, 1 - jõust. 01.04.2017],Maks...",1840
18195,LS,para2,"Käesoleva seaduse tähenduses:,1) anda teed (mi...",https://www.riigiteataja.ee/akt/LS#para2,Mõisted,"Käesoleva seaduse tähenduses:,1) anda teed (mi...",2651
23996,KeTS,para55b1,(1) Maavara kaevandamisõiguse tasust kantakse:...,https://www.riigiteataja.ee/akt/KeTS#para55b1,Laekunud keskkonnatasude jaotumine riigieelarv...,"Maavara kaevandamisõiguse tasust kantakse:,1)...",1450
28406,ESS,para2,Käesolevas seaduses kasutatakse mõisteid järgm...,https://www.riigiteataja.ee/akt/ESS#para2,Mõisted,Käesolevas seaduses kasutatakse mõisteid järgm...,2317
29055,EnKS,para2,Käesolevas seaduses kasutatakse termineid järg...,https://www.riigiteataja.ee/akt/EnKS#para2,Terminid,Käesolevas seaduses kasutatakse termineid järg...,1040
29333,ELTS,para3,"[RT I, 28.06.2012, 1 - jõust. 08.07.2012],Käes...",https://www.riigiteataja.ee/akt/ELTS#para3,Terminid,"[RT I, 28.06.2012, 1 - jõust. 08.07.2012],Käes...",1690
30955,ATKEAS,para24,(1) Maksukohustus tekib aktsiisikauba tarbimis...,https://www.riigiteataja.ee/akt/ATKEAS#para24,Maksukohustuse tekkimine,Käesoleva paragrahvi lõigetes 5–5.2 nimetatud...,1096
30958,ATKEAS,para27,"(1) Aktsiisist on vabastatud:,1) välisriigi di...",https://www.riigiteataja.ee/akt/ATKEAS#para27,Aktsiisivabastus,"Aktsiisist on vabastatud:,1) välisriigi diplo...",1093


In [96]:
GPT_MODEL = "gpt-3.5-turbo"  # only matters insofar as it selects which tokenizer to use
